In [22]:
'''Solving Burgers eqn ut+u*ux=nu*uxx'''
'''I will start with dealiased initial condition and integrate in fourier space to reduce computational cost'''

import numpy as np
from numpy.fft import fft,ifft,fftfreq
import matplotlib.pyplot as plt
from scipy import integrate

In [23]:
#RK-4 method function with parameters iterate of u at present time-upand a get-rhs function f and time  step dt

def rk4(up,f,dt):
    #co-efficients of RK-4 time step
    k1=f(up) 
    k2=f(up+dt*(k1/2))
    k3=f(up+dt*(k2/2))
    k4=f(up+dt*k3)
    un=up+(1/6)*dt*(k1+2*k2+2*k3+k4)
    return un


In [24]:
#checking my rk-4 on a simple ode du/dt=u^(1/3),u(0)=1

dt=0.1 
t=np.arange(0,1,dt) #time points
N=len(t) #numper of time points
actsol=((2*t+3)/3)**(3/2) #actual solution

#applying our created rk-4

compsol=np.zeros_like(t)
compsol[0]=1
for i in range(1,N):
    compsol[i]=rk4(compsol[i-1],lambda x: x**(1/3),dt)

#checking error

error=np.max(np.abs(actsol-compsol))
print(error)

#Ya satisfies at least O(dt**4)

3.5968789280360625e-08


In [25]:
#writing a get-rhs fn of burgers with dealias filter
#parameters -function u,viscosity nu,wavenumber array k,dealias filer dealias

def getrhs(u,nu,k,dealias):
    uk=fft(u) #fft of u
    uxk=1j*k*uk #fft of ux
    ux=np.real(ifft(uxk)) #first space derivative of u
    uxxk=-(k**2)*uk #discrete fourier transform of uxx
    uxx=np.real(ifft(uxxk))
    nlp=u*ux #non-linear term with dealiased u- a blow is happening here with large N,why ?
    nlpk=fft(nlp) #fft of non-linear term
    nlpkde=dealias*nlpk #dealiased fft of nlp
    nlpde=np.real(ifft(nlpkde))
    return -nlpde+nu*uxx
    

In [26]:
#checking if the function works

N=2**15
dx=2*np.pi/N
x=np.arange(0,2*np.pi,dx)
k=fftfreq(N,1/N)
dealias=np.abs(k)<N/3
u=1+np.cos(x)+np.cos(2*x)
ux=-np.sin(x)-2*np.sin(2*x)
uxx=-np.cos(x)-4*np.cos(2*x)
nu=7*(10**(-6))
computedrhs=getrhs(u,nu,k,dealias)
actualrhs=-u*ux+nu*uxx
error=np.max(np.abs(computedrhs-actualrhs))
print(error)

#Yo, not better error :( 

#correct one

2.070255078479022e-11


In [ ]:
#total energy,parameters-u,L,N

def energy(u,L,N):
    dx=L/N #space spacing
    x=np.arange(0,L,dx) #space points
    E=np.abs(u)**2 #velocity squared array
    return integrate.simpson(E,x) #total energy using simpson integral


In [ ]:
#energy spectrum

def energyspectrum(u,k):
    uk=fft(u) #fourier transform of u
    Ek=np.abs(uk)**2 #energy spectrum
    kmax=int(np.max(k)) #max positive-k
    kp=k[0:kmax+1] #positive part of k
    Ek=Ek[0:kmax+1] #Ek for only positive k values
    return kp,Ek


In [ ]:
# transfer function

def transferfunction(u,k,dealias):
    uk=fft(u) #fft of u
    uxk=1j*k*uk #fft of ux
    ux=np.real(ifft(uxk)) #first space derivative of u
    nlp=u*ux #non-linear term with dealiased u- a blow is happening here with large N,why ?
    nlpk=fft(nlp) #fft of non-linear term
    nlpkde=dealias*nlpk #dealiased fft of nlp
    Tk=-nlpkde*np.conj(uk)-np.conj(nlpkde)*uk #the transfer function
    Tk=np.real(Tk)
    kmax=int(np.max(k)) #max in k
    kp=k[0:kmax+1] #positive part of k
    Tkp=Tk[0:kmax+1] #positive part of Tk
    return kp,Tkp
    
    

In [ ]:
#total flux

def flux(u,k,dealias):
    kp,Tkp=transferfunction(u,k,dealias)
    kmax=int(np.max(k))
    pikp=np.zeros_like(kp)
    for i in range(kmax):
        pikp[i]=np.sum(Tkp[i:kmax+1])
    return kp,pikp

In [ ]:

'''solving burgers with periodic boundary conditions using pseudospectral method without dealias and 
plotting total energy and energy spectrum'''

#parameters - interval:[0,L],resolution:N,viscous parameter:nu,maximum time:tmax,timestep:dt,time save point step: s,solution number to be saved:num

def burger_solve(L,N,nu,u0,Tmax,dt,s,no):
    
    #initialization
    
    dx=L/N #space-spacing
    x=np.arange(0,L,dx) #space points
    k=fftfreq(N,1/N)*(2*np.pi)/L #wave numbers
    
    kmax=int(np.max(k)) #max positive-k
    kp=k[0:kmax+1] #positive part of k

    #v=np.vectorize(u0) #vectorizing initial boundary function
    #uin=v(x) #initial function array
    dealias=np.abs(k)<N/3
    
    Nt=round(Tmax/dt) #number of time steps
    t=np.arange(0,Tmax,dt) #time points
    
    #the rk-4 rhs function
    
    def f(u):
        return getrhs(u,nu,k,dealias)
    
    #the rk-4 time stepping
        
    u=u0 #solution array
    
    np.save('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/solution/u'+str(no)+'_0.npy',u)
    
    #energy time series
    
    E=np.zeros_like(t)
    
    E[0]=energy(u,L,N) #initial energy
    
    #k,energy spectrum
    
    Ek=energyspectrum(u,k)
    
    np.save('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/energyspectrum/Ek'+str(no)+'_0.npy',Ek)
    
    #transfer function
    
    Tk=transferfunction(u,k,dealias)
    
    np.save('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/transferfunction/Tk'+str(no)+'_0.npy',Tk)
    
    #derivative of energy spectrum for checking that my Tk term is correct
    
    dEkp=np.zeros_like(kp) #initializing derivative 
    
    #k,flux
    
    pik=flux(u,k,dealias)
    
    np.save('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/flux/pik'+str(no)+'_0.npy',pik)
    
    for i in range(1,Nt):
        
        print(str(i)+" ",end='') #checking that the code is running
        
        u=rk4(u,f,dt) #updated solution
        
        E[i]=energy(u,L,N) #energy at ith timestep
        
        Ekn=energyspectrum(u,k) #new k,energy spectrum
        
        dEkp=(Ekn[1]-Ek[1])/dt #derivative at time (i-1)
        
        Ek=Ekn #updating energy spectrum after computing derivative
        
        Tk=transferfunction(u,k,dealias) # updated k,transferfunction
        
        pik=flux(u,k,dealias) # updated k,flux
        
        #saving u,Ek,pik
        
        if i%s==0:
            
            np.save('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/solution/u'+str(no)+'_'+str(i)+'.npy',u)
            np.save('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/energyspectrum/Ek'+str(no)+'_'+str(i)+'.npy',Ek)
            np.save('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/transferfunction/Tk'+str(no)+'_'+str(i)+'.npy',Tk)
            np.save('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/flux/pik'+str(no)+'_'+str(i)+'.npy',pik)
        
        #saving energy spectrum derivative
            
        if int(i-1)%s==0:
            np.save('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/energyspectrumderivative/dEk'+str(no)+'_'+str(int(i-1))+'.npy',(kp,dEkp))
            
            
    np.save('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/energy/E'+str(no)+'.npy',E)
           
            

In [ ]:

#plotting solution

#parameters -L,N,dt,Tmax,s

def solutionplot(L,N,dt,Tmax,s,no,nu):
    
    dx=L/N #space-spacing
    x=np.arange(0,L,dx) #space points
    k=fftfreq(N,1/N)*(2*np.pi)/L #wave numbers
    
    Nt=round(Tmax/dt) #number of time steps
    t=np.arange(0,Tmax,dt) #time points
    
    for i in range(0,Nt):
        
        print(str(i)+" ",end='')
        
        if i%s==0:
            
            ti=t[i] #present time
            u=np.load('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/solution/u'+str(no)+'_'+str(i)+'.npy')
            
            plt.plot(x,u)
            plt.title('at time '+str(ti)+'nu='+str(nu)+'N='+str(N)+'dt='+str(dt))
            plt.xlabel('x-axis')
            plt.ylabel('solution u(x)')
            plt.savefig('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/solutionplot/u'+str(no)+'_'+str(i)+'.png',dpi=300)
            plt.close()


In [ ]:
#plotting total energy 

def energyplot(L,N,dt,Tmax,s,no,nu):
    
    Nt=round(Tmax/dt) #number of time steps
    t=np.arange(0,Tmax,dt) #time points
    
    E=np.load('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/energy/E'+str(no)+'.npy')
    
    plt.plot(t,E)
    plt.title('nu='+str(nu)+'N='+str(N)+'dt='+str(dt))
    plt.xlabel('time t')
    plt.ylabel('Total energy')
    plt.savefig('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/energyplot/E'+str(no)+'.png',dpi=300)
    

In [ ]:
#plotting energyspectrum

#parameters -L,N,dt,Tmax,s

def energyspectrumplot(L,N,dt,Tmax,s,no,nu):
    
    dx=L/N #space-spacing
    x=np.arange(0,L,dx) #space points
    k=fftfreq(N,1/N)*(2*np.pi)/L #wave numbers
    
    Nt=round(Tmax/dt) #number of time steps
    t=np.arange(0,Tmax,dt) #time points
    
    for i in range(0,Nt):
        
        print(str(i)+" ",end='') #checking run of code
        
        if i%s==0:
            
            ti=t[i] #present time
            kp,Ekp=np.load('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/energyspectrum/Ek'+str(no)+'_'+str(i)+'.npy')
            
            plt.loglog(kp,Ekp,'r')
            plt.loglog(kp,kp**(-2),'b')
            plt.title('at time '+str(ti)+'nu='+str(nu)+'N='+str(N)+'dt='+str(dt))
            plt.xlabel('logk')
            plt.ylabel('logEk')
            #plt.ylim([10**(-7),None])
            plt.savefig('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/energyspectrumplot/Ek'+str(no)+'_'+str(i)+'.png',dpi=300)
            plt.close()
            


In [ ]:
#plotting transfer function and comparing with energyspectrumderivaytive to check accuracy

def transferfunctionplot(L,N,dt,Tmax,s,no,nu):
    
    dx=L/N #space-spacing
    x=np.arange(0,L,dx) #space points
    k=fftfreq(N,1/N)*(2*np.pi)/L #wave numbers
    
    Nt=round(Tmax/dt) #number of time steps
    t=np.arange(0,Tmax,dt) #time points
    
    for i in range(0,Nt):
        
        print(str(i)+" ",end='') #checking run of code
        
        if i%s==0:
            
            ti=t[i] #present time
            kp,Tkp=np.load('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/transferfunction/Tk'+str(no)+'_'+str(i)+'.npy')
            kp,dEkp=np.load('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/energyspectrumderivative/dEk'+str(no)+'_'+str(i)+'.npy')
            
            plt.semilogx(kp,Tkp,label='transfer function')
            plt.semilogx(kp,dEkp,label='energy spectrum derivative')
            plt.legend()
            plt.title('at time '+str(ti)+'nu='+str(nu)+'N='+str(N)+'dt='+str(dt))
            plt.xlabel('logk')
            plt.ylabel('Tk')
            plt.savefig('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/transferfunctionvsenergyspectrumderivativeplot/dEk'+str(no)+'Tk'+str(no)+'_'+str(i)+'.png',dpi=300)
            plt.close()

In [ ]:
#plotting flux

def fluxplot(L,N,dt,Tmax,s,no,nu):
    
    dx=L/N #space-spacing
    x=np.arange(0,L,dx) #space points
    k=fftfreq(N,1/N)*(2*np.pi)/L #wave numbers
    
    Nt=round(Tmax/dt) #number of time steps
    t=np.arange(0,Tmax,dt) #time points
    
    for i in range(0,Nt):
        
        print(str(i)+" ",end='') #checking run of code
        
        if i%s==0:
            
            ti=t[i] #present time
            kp,pikp=np.load('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/flux/pik'+str(no)+'_'+str(i)+'.npy')
            
            plt.semilogx(kp,pikp)
            plt.title('at time '+str(ti)+'nu='+str(nu)+'N='+str(N)+'dt='+str(dt))
            plt.xlabel('logk')
            plt.ylabel('pik')
            plt.savefig('/localhome/souryajit/Desktop/mycodes/fftcodes/burgersrandomplots/fluxplot/pik'+str(no)+'_'+str(i)+'.png',dpi=300)
            plt.close()

In [ ]:
#running the solver for inviscid/viscous

L=2*np.pi
N=2**9
nu=10**(-2)
dx=L/N
x=np.arange(0,L,dx)
u0=1+np.cos(x)+np.cos(2*x)

Tmax=2
dt=10**(-3)
s=10**2


burger_solve(L,N,nu,u0,Tmax,dt,s)



In [ ]:
#running solver for random initialization for first 6 wave numbers

L=2*np.pi

N=2**15

nu=4*10**(-4)

k=fftfreq(N,1./N)

#fft of inital data

u0k=np.zeros_like(k,dtype='complex_')

rand_array_real=np.random.rand(6)

rand_array_imaginary=np.random.rand(6)

u0k[0]=rand_array_real[0]+1j*rand_array_imaginary[0]

for i in range(1,6):
    u0k[i]=rand_array_real[i]+1j*rand_array_imaginary[i]
    u0k[N-i]=rand_array_real[i]-1j*rand_array_imaginary[i]
    
u0k=N*u0k
    
#initial solution

u0=np.real(ifft(u0k))

Tmax=2

dt=10**(-5)

s=10**4

no=18

burger_solve(L,N,nu,u0,Tmax,dt,s,no)


In [ ]:
#plotting solution for inviscid/viscous

L=2*np.pi
N=2**15
Tmax=2
dt=10**(-5)
s=10**4
nu=4*10**(-4)
no=18

solutionplot(L,N,dt,Tmax,s,no,nu)
    


In [ ]:
#plotting energy time series for inviscid/viscous

L=2*np.pi
N=2**15
Tmax=2
dt=10**(-5)
s=10**4
nu=4*10**(-4)
no=18
energyplot(L,N,dt,Tmax,s,no,nu)



In [ ]:
#plotting energyspectrum for inviscid/viscous

L=2*np.pi
N=2**15
Tmax=2
dt=10**(-5)
s=10**4
nu=4*10**(-4)
no=18

energyspectrumplot(L,N,dt,Tmax,s,no,nu)



In [ ]:
#plotting transfer function vs energy spectrum derivative for inviscid case to check accuracy

L=2*np.pi
N=2**15
Tmax=2
dt=10**(-5)
s=10**4
nu=4*10**(-4)
no=18


transferfunctionplot(L,N,dt,Tmax,s,no,nu)

'''initially there is an error in the small w nos for time t=0 which I think because 
we multiplied a dealias to the non linear term'''

In [ ]:
#plotting flux for inviscid/viscous

L=2*np.pi
N=2**15
Tmax=2
dt=10**(-5)
s=10**4
nu=4*10**(-4)
no=18


fluxplot(L,N,dt,Tmax,s,no,nu)




In [ ]:
%timeit burger_solve